**Table of contents**<a id='toc0_'></a>    
- 1. [Function definitions](#toc1_)    
- 2. [load pkl files](#toc2_)    
- 3. [concatenate df's](#toc3_)    
- 4. [Exploratory Analysis](#toc4_)    
- 5. [Data Cleaning](#toc5_)    
- 6. [Extract fit_param into dict](#toc6_)    
- 7. [select channel](#toc7_)    
- 8. [Investigating Paramater distributions](#toc8_)    
  - 8.1. [$\sigma$](#toc8_1_)    
    - 8.1.1. [fitting](#toc8_1_1_)    
  - 8.2. [$\tau$](#toc8_2_)    
  - 8.3. [scale](#toc8_3_)    
- 9. [Cuts:  rectangular cut](#toc9_)    
  - 9.1. [$ \sigma$](#toc9_1_)    
    - 9.1.1. [fitting](#toc9_1_1_)    
  - 9.2. [$\tau$](#toc9_2_)    
    - 9.2.1. [fitting](#toc9_2_1_)    
    - 9.2.2. [Fitting with iMinuit](#toc9_2_2_)    
  - 9.3. [scale](#toc9_3_)    
    - 9.3.1. [fitting](#toc9_3_1_)    
- 10. [FingerPlot](#toc10_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
import sys
# import os
import numpy as np
import pandas as pd
import pickle
# from scipy.stats import chisquare
# from scipy.integrate import simpson

# import matplotlib
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
# from matplotlib.ticker import AutoMinorLocator
# from matplotlib.ticker import FormatStrFormatter

# from termcolor import colored
# from tqdm.notebook import tqdm
# from scipy.ndimage import gaussian_filter
from scipy.optimize import curve_fit

In [ ]:
import matplotlib.patches as patches

In [ ]:
np.set_printoptions(formatter={'float': lambda x: f"{x:10.4g}"})

In [ ]:
%matplotlib ipympl 

# 1. <a id='toc1_'></a>[Function definitions](#toc0_)

In [ ]:
def f_gauss(x, g_mean, g_sigma, g_k):
    return g_k*(1/(g_sigma*np.sqrt(2*np.pi)))*np.exp(-0.5*((x-g_mean)/g_sigma)**2)

In [ ]:
def red_chisq(f_obs, f_exp, fittedparameters):
    chisqr = np.sum((f_obs - f_exp)**2 / f_exp)
    ndf = f_obs.shape[0]
    return chisqr/(ndf -fittedparameters.shape[0])

# 2. <a id='toc2_'></a>[load pkl files](#toc0_)

In [ ]:
# all_clean_catalogues = []
# for run_number in run_number_ls:
#     all_clean_catalogues.append(f'clean_catalogue_run000{run_number}.pkl')

In [ ]:
# selected_clean_catalogue_path = os.path.join(output_folder, all_clean_catalogues[3])
# selected_clean_dict = pickle.load(open(selected_fit_catalogue_path, 'rb'))
# selected_clean_dict #['ch2']

In [ ]:
# output_folder = '/home/sarthak/my_projects/argset/output_folder/'
output_folder = '/work/chuck/sarthak/argset/output_folder/'
# output_folder = '/work/sarthak/argset/processed_data/backup_ch2/'
# output_folder = '/work/sarthak/argset/processed_data/backup/'

In [ ]:
run_number_ls = [52, 53, 54, 61, 62, 63]
# run_number_ls = [52, 61]

In [ ]:
all_fit_catalogues = []
for run_number in run_number_ls:
    all_fit_catalogues.append(f'{output_folder}/fit_catalogue_run000{run_number}.pkl')
    # all_fit_catalogues.append(f'{output_folder}/fit_catalogue_custom_run000{run_number}.pkl')

In [ ]:
all_fit_catalogues

In [ ]:
fit_dict_52 = pickle.load(open( all_fit_catalogues[0], 'rb'))

In [ ]:
fit_dict_53 = pickle.load(open( all_fit_catalogues[1], 'rb'))

In [ ]:
fit_dict_54 = pickle.load(open( all_fit_catalogues[2], 'rb'))

In [ ]:
fit_dict_61 = pickle.load(open( all_fit_catalogues[3], 'rb'))

In [ ]:
fit_dict_62 = pickle.load(open( all_fit_catalogues[4], 'rb'))

In [ ]:
fit_dict_63 = pickle.load(open( all_fit_catalogues[4], 'rb'))

# 3. <a id='toc3_'></a>[concatenate df's](#toc0_)

In [ ]:
fit_dict_ls = [fit_dict_52, fit_dict_53, fit_dict_54, fit_dict_61, fit_dict_62, fit_dict_63]
# fit_dict_ls = [fit_dict_61, fit_dict_62, fit_dict_63]
ch_str_ls = ['ch0', 'ch1', 'ch2']
# ch_str_ls = ['ch2']

fit_catalogue = {}
for ch_x in ch_str_ls:
    fit_catalogue_x = []
    for fit_dict in fit_dict_ls:
        fit_catalogue_x.append(fit_dict[ch_x])
    fit_catalogue[ch_x] = pd.concat(fit_catalogue_x, ignore_index=True)

<p> Note: remember event_counter is unique only for a run; for concatenated DataFrames it is no longer unique identifier.</p>

# 4. <a id='toc4_'></a>[Exploratory Analysis](#toc0_)

the ratio of good fit waveforms to clean forms

In [ ]:
# (fit_catalogue['ch0'][fit_catalogue['ch0']['chisqr']< 2.0]).shape[0]/(fit_catalogue['ch0']).shape[0]

In [ ]:
# (fit_catalogue['ch1'][fit_catalogue['ch1']['chisqr']< 2.0]).shape[0]/(fit_catalogue['ch1']).shape[0]

In [ ]:
# (fit_catalogue['ch2'][fit_catalogue[ch_x]['chisqr']< 2.0]).shape[0]/(fit_catalogue['ch2']).shape[0]

How many times fit failed to converge?

In [ ]:
# fit_catalogue['ch0']['fit_param'].isnull().sum()

In [ ]:
# fit_catalogue['ch1']['fit_param'].isnull().sum()

In [ ]:
# fit_catalogue['ch2']['fit_param'].isnull().sum()

In [ ]:
# fit_catalogue['ch0'].index.values.shape

In [ ]:
# fit_catalogue['ch0'][fit_catalogue['ch0']['chisqr']< 2.0]

# 5. <a id='toc5_'></a>[Data Cleaning](#toc0_)

Drop NA rows

In [ ]:
for ch_index in ch_str_ls:
    fit_catalogue[ch_index] = fit_catalogue[ch_index].dropna()

In [ ]:
# sanity check: dropna works!
# fit_catalogue['ch0']['fit_param'].isnull().sum()

Histogram

# 6. <a id='toc6_'></a>[Extract fit_param into dict](#toc0_)

extracting param values

In [ ]:
param_dict = {}
for ch_index in ch_str_ls:
    t0_ls = []
    sigma_ls = []
    tau_ls = []
    scale_ls = []
    fit_param_x = {}
    for loop_index in fit_catalogue[ch_index].index.values:
        loop_fit_param = fit_catalogue[ch_index].loc[loop_index]['fit_param']
        t0_ls.append(loop_fit_param[0])
        sigma_ls.append(loop_fit_param[1])
        tau_ls.append(loop_fit_param[2])
        scale_ls.append(loop_fit_param[3])
    fit_param_x['t0'] = t0_ls
    fit_param_x['sigma'] = sigma_ls
    fit_param_x['tau'] = tau_ls
    fit_param_x['scale'] = scale_ls
    param_dict[ch_index] = fit_param_x

In [ ]:
# fit_catalogue['ch0'].index.values.shape

In [ ]:
# n_param = fit_catalogue['ch0']['fit_param'][0].shape[0]

# 7. <a id='toc7_'></a>[select channel](#toc0_)

Which channel?

In [ ]:
# selected_channel =  input("Choose channel number for processing.")
selected_channel = 2
print(f"selected channel index: {selected_channel}")
ch_x = ch_str_ls[int(selected_channel)]
### ch_str_ls = ['ch0', 'ch1', 'ch2']
### ch_x = ch_str_ls[2]

In [ ]:
plt.close(1)
fig1, ax1 = plt.subplots()
text_in_box = AnchoredText(f"statistics = {len(param_dict[ch_x]['t0'])}", \
                                       loc='upper right')
ax1.add_artist(text_in_box)
ax1.hist(param_dict[ch_x]['t0'], bins = np.linspace(350, 550, 200))
ax1.set_title('t0')

# 8. <a id='toc8_'></a>[Investigating Paramater distributions](#toc0_)

## 8.1. <a id='toc8_1_'></a>[$\sigma$](#toc0_)

In [ ]:
plt.close(2)
fig2, ax2 = plt.subplots()
hist_content, hist_edges, _hist_obj= ax2.hist(param_dict[ch_x]['sigma'], bins=200, histtype='step', label = 'data') #, bins= np.linspace(1.0, 6.0, 10))
text_in_box = AnchoredText(f"statistics = {len(param_dict[ch_x]['sigma'])}", \
                                       loc='upper right')
ax2.add_artist(text_in_box)
ax2.set_title('$\\sigma$')

### 8.1.1. <a id='toc8_1_1_'></a>[fitting](#toc0_)

In [ ]:
x_range = range(30, 49)

# p0_input = [2, 0.45, 500.0] ## ch2
p0_input = [2, 0.45, 500.0] ## ch1
# bounds_input = ([1.0, 0.0, 500.0], [10.0, 1.0, 600.0]) ## ch2
bounds_input = ([1.0, 0.0, 0.0], [10.0, 1.0, np.inf]) ## ch1
fitted_parameters, _pcov = curve_fit(f_gauss, 
                            hist_edges[x_range], hist_content[x_range], \
                            p0 = p0_input, 
                            bounds = bounds_input,
                            sigma = np.sqrt(hist_content[x_range]),
                            absolute_sigma=True,
                            )

chisqr_value = red_chisq(hist_content[x_range], \
    f_gauss(hist_edges[x_range], *fitted_parameters), fitted_parameters
    )

chisqr_value

In [ ]:
fitted_parameters

In [ ]:
ax2.plot(hist_edges[x_range], f_gauss(hist_edges[x_range], *fitted_parameters), 
        #  color='red', 
         label='fitted function')
ax2.legend()
chisqr_in_box = AnchoredText(f"red. chisqr = {chisqr_value:.2f}", \
                                       loc='center right')
ax2.add_artist(chisqr_in_box)

## 8.2. <a id='toc8_2_'></a>[$\tau$](#toc0_)

In [ ]:
plt.close(3)
fig3, ax3 = plt.subplots()
ax3.hist = plt.hist(param_dict[ch_x]['tau'], bins= np.linspace(0.0, 150.0, 300), histtype='step')
text_in_box = AnchoredText(f"statistics = {len(param_dict[ch_x]['tau'])}", \
                                       loc='upper right')
ax3.add_artist(text_in_box)
ax3.set_title('$\\tau$')

## 8.3. <a id='toc8_3_'></a>[scale](#toc0_)

In [ ]:
plt.close(4)
fig4, ax4 = plt.subplots()
ax4.hist(param_dict[ch_x]['scale'], bins= np.linspace(0.4, 1.0, 200), histtype='step')
text_in_box = AnchoredText(f"statistics = {len(param_dict[ch_x]['scale'])}", \
                                       loc='upper left')
ax4.add_artist(text_in_box)
ax4.set_title('scale')

In [ ]:
plt.close(5)

red_win = patches.Rectangle((0.932, 2.44), 0.06, 3.10, linewidth=1.1, edgecolor='r', facecolor='none')
fig5, ax5 = plt.subplots()
hist_h, hist_xedges, hist_yedges, hist_im = ax5.hist2d(param_dict[ch_x]['scale'], \
                    param_dict[ch_x]['sigma'], bins = [np.linspace(0.6, 1.0, 100), np.linspace(0., 10., 100)], \
                    cmap='jet')
ax5.add_patch(red_win)
ax5.set_xlabel('scale')
ax5.set_ylabel('$\sigma$')
fig5.colorbar(hist_im)
ax5.set_title(f' {ch_x}: $scale\ vs \ \\sigma$');

In [ ]:
plt.close(6)
fig6, ax6 = plt.subplots()
hist_h, hist_xedges, hist_yedges, hist_im = ax6.hist2d(param_dict[ch_x]['sigma'], \
                    param_dict[ch_x]['tau'], bins = [np.linspace(0.0, 1.0, 10), np.linspace(50e0, 20e3, 10)] , \
                        cmap='jet')
fig6.colorbar(hist_im)
ax6.set_title('$\\sigma \ vs \ \\tau $');

In [ ]:
plt.close(7)
fig7, ax7 = plt.subplots()
hist_h, hist_xedges, hist_yedges, hist_im = ax7.hist2d(param_dict[ch_x]['tau'], \
                    param_dict[ch_x]['scale'], bins = [np.linspace(50, 175, 1000), np.linspace(0.5, 1.0, 10)], \
                    cmap='jet')
fig7.colorbar(hist_im)
ax7.set_title('$\\tau \ vs \ scale $');

# 9. <a id='toc9_'></a>[Cuts:  rectangular cut](#toc0_)

<span style='color:lightblue'> List of events to be removed</span>

In [ ]:
### list of events which contribute to red window for Ch2
internal_points = []
for fit_index in fit_catalogue[ch_x].index.values:
    loop_fit_param = fit_catalogue[ch_x]['fit_param'].loc[fit_index]
    param_point = (loop_fit_param[3], loop_fit_param[1]) # scale, sigma
    transformed_interior_point = red_win.get_data_transform().transform(param_point)
    if red_win.contains_point(transformed_interior_point, radius=0):
        internal_points.append(fit_index)

<span style='color:lightblue'> applying cut </span>

In [ ]:
fit_catalogue[ch_x] = fit_catalogue[ch_x].drop(index = internal_points)

<span style='color:lightblue'> extracting param_dict out of DataFrames for all channels </span>

In [ ]:
param_dict_cut = {}
for ch_index in ch_str_ls:
# for ch_x in ['ch2']: #ch_str_ls:
    t0_ls = []
    sigma_ls = []
    tau_ls = []
    scale_ls = []
    fit_param_x = {}
    for loop_index in fit_catalogue[ch_index].index.values:
        loop_fit_param = fit_catalogue[ch_index].loc[loop_index]['fit_param']
        t0_ls.append(loop_fit_param[0])
        sigma_ls.append(loop_fit_param[1])
        tau_ls.append(loop_fit_param[2])
        scale_ls.append(loop_fit_param[3])
    fit_param_x['t0'] = t0_ls
    fit_param_x['sigma'] = sigma_ls
    fit_param_x['tau'] = tau_ls
    fit_param_x['scale'] = scale_ls
    param_dict_cut[ch_index] = fit_param_x

In [ ]:
# selected_channel = input("enter channel number")
ch_x = ch_str_ls[int(selected_channel)]
# ch_x = 'ch2'

In [ ]:
plt.close(22)
fig22, ax22 = plt.subplots()
text_in_box = AnchoredText(f"statistics = {len(param_dict_cut[ch_x]['t0'])}", \
                                       loc='upper right')
ax22.add_artist(text_in_box)
ax22.hist(param_dict_cut[ch_x]['t0'], bins = np.linspace(350, 550, 100))
ax22.set_title(f"{ch_x} t0: patch removed")

## 9.1. <a id='toc9_1_'></a>[$ \sigma$](#toc0_)

In [ ]:
plt.close(23)
fig23, ax23 = plt.subplots()
hist_content, hist_edges, _hist_obj= ax23.hist(param_dict_cut[ch_x]['sigma'], bins=200, histtype='step', label='sigma') #, bins= np.linspace(1.0, 6.0, 10))
# text_in_box = AnchoredText(f"statistics = {len(param_dict_cut[ch_x]['sigma'])}", \
                                    #    loc='lower right')
# ax23.add_artist(text_in_box)
ax23.set_title(f"{ch_x} $\\sigma$: patch removed")
## error bars
hist_centers = []
for i in range(hist_edges.shape[0]):
    center_variable = np.mean(hist_edges[i:i+2])
    hist_centers.append(center_variable)

hist_centers = np.array(hist_centers)

ax23.errorbar(hist_centers[:hist_content.shape[0]], hist_content, \
              yerr = np.sqrt(hist_content), elinewidth=1.0, linestyle='none', ecolor='green')

### 9.1.1. <a id='toc9_1_1_'></a>[fitting](#toc0_)

In [ ]:
# x_range # ch1 & ch2
x_range = range(24, 40) ##ch0

In [ ]:
p0_input = [2, 0.45, 500.0]
bounds_input = ([1.50, 0.0, 0.0], [2.15, 1.0, np.inf])

fitted_parameters_sigma, _pcov = curve_fit(f_gauss, 
                            hist_edges[x_range], hist_content[x_range], \
                            p0 = p0_input, 
                            sigma = np.sqrt(hist_content[x_range]),
                            absolute_sigma=True,
                            bounds = bounds_input,
                            )

ax23.plot(hist_edges[x_range], f_gauss(hist_edges[x_range], *fitted_parameters_sigma), \
          color='red', label='fitted function')

ax23.legend()

chisqr_value = red_chisq(hist_content[x_range], \
    f_gauss(hist_edges[x_range], *fitted_parameters_sigma), fitted_parameters_sigma
    )

text_in_box = AnchoredText(f"statistics = {len(param_dict_cut[ch_x]['sigma'])} \nred. chisqr = {chisqr_value:.2f}", \
                                       loc='center right')
ax23.add_artist(text_in_box)

fitted_parameters_sigma

## 9.2. <a id='toc9_2_'></a>[$\tau$](#toc0_)

In [ ]:
# ch_x = 'ch2'
plt.close(24)
fig24, ax24 = plt.subplots()
hist_content, hist_edges, _hist_obj = ax24.hist(param_dict_cut[ch_x]['tau'], bins= np.linspace(0.0, 150.0, 250), histtype='step')
# text_in_box = AnchoredText(f"statistics = {len(param_dict_cut[ch_x]['tau'])}", \
#                                        loc='upper right')
# ax24.add_artist(text_in_box)
ax24.set_title(f"{ch_x} $\\tau$: patch removed")

##error bars
hist_centers = []
for i in range(hist_edges.shape[0]):
    center_variable = np.mean(hist_edges[i:i+2])
    hist_centers.append(center_variable)

hist_centers = np.array(hist_centers)

ax24.errorbar(hist_centers[:hist_content.shape[0]], hist_content, yerr = np.sqrt(hist_content), elinewidth=1.0, linestyle='none', ecolor='green')

### 9.2.1. <a id='toc9_2_1_'></a>[fitting](#toc0_)

In [ ]:
# x_range = range(58, 91) # ch2
# x_range = range(58, 91) # ch1
x_range = range(58, 91) # ch0

In [ ]:
p0_input = [50.0, 9.0, 8000.0] # ch1 & ch2
# bounds_input = ([20.0, 0.05, 1.0], [60.0, 15.5, 10_000]) ## ch2
# bounds_input = ([20.0, 0.05, 1.0], [60.0, 15.5, np.inf]) ## ch1
bounds_input = ([20.0, 0.05, 1.0], [60.0, 15.5, np.inf]) ## ch0

fitted_parameters_tau, _pcov = curve_fit(f_gauss, \
                            hist_edges[x_range], hist_content[x_range], \
                            p0 = p0_input, \
                            bounds = bounds_input, \
                            sigma = np.sqrt(hist_content[x_range]), \
                            absolute_sigma=True, \
                            # x_scale = [10, 1, 1000], \
                            method = 'trf', \
                            ## method = 'dogbox', \
                            )

### plotting
ax24.plot(hist_edges[x_range], f_gauss(hist_edges[x_range], *fitted_parameters_tau), color='red', label='fitted function')
ax24.legend(loc='upper right')

chisqr_value = red_chisq(hist_content[x_range], \
    f_gauss(hist_edges[x_range], *fitted_parameters_tau), fitted_parameters_tau
    )

text_in_box = AnchoredText(f"statistics = {len(param_dict_cut[ch_x]['tau'])} \nred. chisqr = {chisqr_value:.2f}", \
                                       loc='center right')

ax24.add_artist(text_in_box)

In [ ]:
fitted_parameters_tau

### 9.2.2. <a id='toc9_2_2_'></a>[Fitting with iMinuit](#toc0_)

In [ ]:
# import iminuit
# from iminuit.cost import LeastSquares
# from iminuit import Minuit

In [ ]:
# least_squares = LeastSquares(x_values[58:91], hist_content[58:91], np.sqrt(hist_content[58:91]), f_gauss)
# # least_squares = LeastSquares(x_values, hist_content[:x_values.shape[0]], np.ones(x_values.shape[0]), f_gauss)

In [ ]:
# m_obj = Minuit(least_squares, g_mean = 50.0, g_sigma = 9.5, g_k = 8000)

In [ ]:
# m_obj.simplex()

In [ ]:
# m_obj.migrad()

## 9.3. <a id='toc9_3_'></a>[scale](#toc0_)

In [ ]:
# ch_x = 'ch2'
plt.close(25)
fig25, ax25 = plt.subplots()
hist_content, hist_edges, _hist_obj = ax25.hist(param_dict_cut[ch_x]['scale'], \
                                                bins= np.linspace(0.4, 1.0, 200), histtype='step', label='data')
# text_in_box = AnchoredText(f"statistics = {len(param_dict_cut[ch_x]['scale'])}", \
#                                        loc='upper left')
# ax25.add_artist(text_in_box)
ax25.set_title(f"{ch_x} scale: patch removed")

### 9.3.1. <a id='toc9_3_1_'></a>[fitting](#toc0_)

In [ ]:
# x_range = range(130, 172) #range(130, 177) 160 165 # ch2
x_range = range(153, 172) # ch0

In [ ]:
p0_input = [0.89, 0.09, 200.0]
bounds_input = ([0.8, 0.01, 1.0], [1.0, 1.0, 10_000])

fitted_parameters_scale, _pcov = curve_fit(f_gauss, \
                            hist_edges[x_range], hist_content[x_range], \
                            p0 = p0_input, \
                            bounds = bounds_input, \
                            sigma = np.sqrt(hist_content[x_range]), \
                            absolute_sigma=True, \
                            # x_scale = [10, 1, 1000], \
                            method = 'trf', \
                            ## method = 'dogbox', \
                            )
##error bars
hist_centers = []
for i in range(hist_edges.shape[0]):
    center_variable = np.mean(hist_edges[i:i+2])
    hist_centers.append(center_variable)

hist_centers = np.array(hist_centers)

ax25.errorbar(hist_centers[:hist_content.shape[0]], hist_content, \
              yerr = np.sqrt(hist_content), elinewidth=1.0, linestyle='none', ecolor='green')
### plotting
ax25.plot(hist_edges[x_range], f_gauss(hist_edges[x_range], *fitted_parameters_scale), \
          color='red', label='fitted function')
ax25.legend(loc='lower left')

chisqr_value = red_chisq(hist_content[x_range], \
    f_gauss(hist_edges[x_range], *fitted_parameters_scale), fitted_parameters_scale
    )

print(f'chisqr_value: {chisqr_value}')
print(f'fitted_parameters_scale: {fitted_parameters_scale}')

text_in_box = AnchoredText(f"statistics = {len(param_dict_cut[ch_x]['scale'])} \nred. chisqr = {chisqr_value:.2f}", \
                                       loc='center left')

ax25.add_artist(text_in_box)

2d parameter space 

In [ ]:
# ch_x = 'ch2'
plt.close(26)
fig26, ax26 = plt.subplots()
red_win = patches.Rectangle((0.932, 2.44), 0.06, 3.10, linewidth=1.1, edgecolor='r', facecolor='none')

hist_h, hist_xedges, hist_yedges, hist_im = ax26.hist2d(param_dict_cut[ch_x]['scale'], param_dict_cut[ch_x]['sigma'], \
           bins = [np.linspace(0.6, 1.0, 100), np.linspace(0., 10., 100)], cmap='jet')
ax26.add_patch(red_win)
ax26.set_xlabel('scale')
ax26.set_ylabel('$\sigma$')
fig26.colorbar(hist_im)
ax26.set_title(f"{ch_x} $scale\ vs \ \\sigma$: patch removed");

In [ ]:
# ch_x = 'ch2'
plt.close(27)
fig27, ax27 = plt.subplots()
hist_h, hist_xedges, hist_yedges, hist_im = ax27.hist2d(param_dict_cut[ch_x]['sigma'], param_dict_cut[ch_x]['tau'], \
           bins = [np.linspace(0.0, 1.0, 10), np.linspace(50e0, 20e3, 10)] , cmap='jet')
ax27.set_xlabel('scale')
ax27.set_ylabel('$\sigma$')
fig27.colorbar(hist_im)
ax27.set_title(f"{ch_x} $\\sigma \ vs \ \\tau $: patch removed");

In [ ]:
# ch_x = 'ch2'
plt.close(28)
fig28, ax28 = plt.subplots()
hist_h, hist_xedges, hist_yedges, hist_im = ax28.hist2d(param_dict_cut[ch_x]['tau'], \
            param_dict_cut[ch_x]['scale'], bins = [np.linspace(50, 175, 1000), np.linspace(0.5, 1.0, 10)] ,\
            cmap='jet')

ax28.set_xlabel('scale')
ax28.set_ylabel('$\sigma$')
fig28.colorbar(hist_im)
ax28.set_title(f"{ch_x} $\\tau \ vs \ scale $: patch removed");

# 10. <a id='toc10_'></a>[FingerPlot](#toc0_)

In [ ]:
# fit_catalogue[ch_x]

In [ ]:
(fit_catalogue[ch_x]['chisqr']<2).sum()

In [ ]:
# pulse_integration_window_begin = 450
# pulse_integration_window_end = 650

integration_vector = np.array([])

for wf_index in fit_catalogue[ch_x]['wf_raw'].index.values:
    # if fit_catalogue[ch_x]['chisqr'].loc[wf_index] < 2.0:
    if True:
        fitted_param = fit_catalogue[ch_x]['fit_param'][wf_index]
        fitted_baseline = fitted_param[4]
        trace = fit_catalogue[ch_x]['wf_raw'].loc[wf_index] - fitted_baseline
        peak_loc =  int(fitted_param[0])
        # pulse_win_integration_value = np.sum(trace[pulse_integration_window_begin:pulse_integration_window_end]) # fixed window
        pulse_win_integration_value = np.sum(trace[peak_loc-10:peak_loc+50]) # asymmetric window around peak
        integration_vector = np.append(integration_vector, pulse_win_integration_value)

In [ ]:
fig99, ax99 = plt.subplots()
# ax99.hist(integration_vector, bins=np.arange(0, 6000, 10), color='C0', label='ch2'); # fixed window
ax99.hist(integration_vector, bins=np.arange(0, 2500, 10), color='C0', label=ch_x); # peak_loc works for ch1
# ax99.hist(integration_vector, bins=np.arange(0, 2500, 10), color='C0', label=ch_x); # peak_loc works for ch1
# ax99.hist(integration_vector, bins=np.arange(0, 10000, 25), color='C0', label=ch_x); # peak_loc works for ch0
# ax99.hist(integration_vector, bins=2000, color='C0', label='ch2');
ax99.legend()
ax99.set_title(f'Finger plot for {ch_x}')
ax99.set_xlabel('integrated ADC counts equivalent to charge')
ax99.set_ylabel('histogram frequency')
text_in_box = AnchoredText(f"statistics = {len(integration_vector)}", \
                                       loc='center right')
ax99.add_artist(text_in_box)

In [ ]:
sys.exit()

Fingerplot from pulse model

In [ ]:
def pulse_template(t, t0, sigma, tau, scale, baseline, K) -> np.ndarray:
    ''' 
    ARMA template.
    t0 - offset
    K - scalar multiplication factor for matching data
    '''
    return baseline + K*((1-scale)/(sigma*np.sqrt(2*np.pi))*np.exp(-((t-t0)/(sigma))**2/2) + scale*np.heaviside(t-t0,0)/tau*np.exp(-(t-t0)/tau))

In [ ]:
x_values = np.arange(fit_catalogue[ch_x]['wf_raw'][0].shape[0])

In [ ]:
fit_catalogue[ch_x]['wf_raw'].index.values

In [ ]:
wf_index = 63688
fig111, ax111 = plt.subplots()
fitted_param = fit_catalogue[ch_x]['fit_param'][wf_index]
ax111.plot(pulse_template(x_values, *fitted_param) - fitted_param[4])

In [ ]:
from scipy.integrate import simpson

In [ ]:
integration_vector = np.array([])
for wf_index in fit_catalogue[ch_x]['wf_raw'].index.values:
    if fit_catalogue[ch_x]['chisqr'].loc[wf_index] < 2:
        fitted_param = fit_catalogue[ch_x]['fit_param'][wf_index]
        trace = pulse_template(x_values, *fitted_param) - fitted_param[4]
        # pulse_win_integration_value = np.sum(trace)
        pulse_win_integration_value = simpson(trace)
        integration_vector = np.append(integration_vector, pulse_win_integration_value)

In [ ]:
fig111, ax111 = plt.subplots()
# ax111.hist(integration_vector, bins=np.arange(450, 6000, 60), color='C0', label='ch2'); # fixed window
ax111.hist(integration_vector, bins=np.arange(0, 2000, 15), color='C0', label=f'{ch_x}'); # peak_loc
# ax111.hist(integration_vector, bins=2000, color='C0', label='ch2');
ax111.legend()
ax111.set_title(f'{ch_x}: Finger plot from pulse model')
ax111.set_xlabel('integrated ADC counts equivalent to charge')
ax111.set_ylabel('histogram frequency')
text_in_box = AnchoredText(f"statistics = {len(integration_vector)}", \
                                       loc='upper left')
ax111.add_artist(text_in_box)

<hr> </hr>